<a href="https://colab.research.google.com/github/mandar-solanki/agents-course-ms/blob/main/unit2/smolagents/multiagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet smolagents -U
!pip install --quiet ddgs
!pip install --quiet 'smolagents[litellm]'
!pip install --quiet 'smolagents[openai]'
!pip install --quiet plotly geopandas shapely kaleido -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 4.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os
llm_model_name1 = 'gemini/gemini-2.0-flash'
llm_model_name2 = 'gemini/gemini-2.0-flash-lite'
api_key = userdata.get("GEMINI_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool
from smolagents import LiteLLMModel, DuckDuckGoSearchTool, OpenAIServerModel

# model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
model1 = LiteLLMModel(
    model_id=llm_model_name1,
    # api_key=api_key,
    max_tokens=8096
)

model2 = LiteLLMModel(
    model_id=llm_model_name2,
    # api_key=api_key,
    max_tokens=8096
)

big_model = OpenAIServerModel(
    model_id='gemini-2.0-flash',
    api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=api_key
)

In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here
(we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
agent = CodeAgent(
    model=model2,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)


In [ ]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  from geopy.geocoders import Nominatim                                                                            
  from geopy.distance import great_circle                                                                          
                                                                                                                   
  locations_search_results = web_search("Batman filming locations")                                                
  print(locations_search_results)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'from geopy.geocoders import Nominatim' due to: InterpreterError: Import from 
geopy.geocoders is not allowed. Authorized imports are: ['random', 'pandas', 'unicodedata', 'datetime', 
'statistics', 'itertools', 'math', 're', 'time', 'queue', 'stat', 'collections'\]

[Step 1: Duration 2.18 seconds| Input tokens: 2,366 | Output tokens: 130]

...

[Step 6: Duration 0.83 seconds| Input tokens: 26,450 | Output tokens: 1,086]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Batman Filming Locations:\n" + df.to_string() + "\n\nSupercar Factories:\n" +                      
  supercar_df.to_string() + "\n\nMcLaren factory in Woking, UK has similar transfer time to Gotham as some Batman  
  filming locations (London).")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Batman Filming Locations:
             travel_time_hours
London                    9.19
Chicago                   2.68
Liverpool                 8.81
Glasgow                   8.60
Los Angeles               6.77

Supercar Factories:
             travel_time_hours
Lamborghini              10.79
Koenigsegg               10.11
Ferrari                  10.73
McLaren                   9.14

McLaren factory in Woking, UK has similar transfer time to Gotham as some Batman filming locations (London).

[Step 7: Duration 1.38 seconds| Input tokens: 33,323 | Output tokens: 1,213]

In [ ]:
print(result)

Batman Filming Locations:
             travel_time_hours
London                    9.19
Chicago                   2.68
Liverpool                 8.81
Glasgow                   8.60
Los Angeles               6.77

Supercar Factories:
             travel_time_hours
Lamborghini              10.79
Koenigsegg               10.11
Ferrari                  10.73
McLaren                   9.14

McLaren factory in Woking, UK has similar transfer time to Gotham as some Batman filming locations (London).


In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
*   Our current location is Gotham (40.7128° N, 74.0060° W).
*   The task requires finding Batman filming locations.
*   The task requires calculating cargo plane travel time.
*   The task requires finding supercar factories.
*   The result must be a pandas dataframe.

### 1.2. Facts to look up
*   Batman filming locations. Sources to consider: Websites listing movie filming locations (e.g., IMDb, movie fan 
sites).
*   Coordinates (latitude, longitude) for each Batman filming location.
*   Supercar factories. Sources to consider: Websites listing supercar manufacturers.
*   Coordinates (latitude, longitude) for each supercar factory.
*   Cargo plane cruising speed (if a default value is needed, or verify it).

### 1.3. Facts to derive
*   Calculate the travel time from Gotham to each Batman filming location using the function 
`calculate_cargo_travel_time`.
*   Calculate the travel time from Gotham to each supercar factory using the function 
`calculate_cargo_travel_time`.
*   Compare travel times between Batman filming locations and supercar factories.
*   Create a pandas dataframe with the Batman filming locations and their travel times.
*   Filter supercar factories to those with roughly similar travel times.

## 2. Plan

1.  **Search for Batman filming locations:** Use a web search to find websites listing Batman filming locations.
2.  **Visit each relevant website:** Iterate through the search results, visiting the websites that seem promising.
3.  **Extract filming locations and their coordinates:** Extract the location names and associated coordinates from
each visited website.
4.  **Verify coordinates (if not provided):** For locations without readily available coordinates, use a web search
to find their latitude and longitude.
5.  **Calculate travel times to Batman locations:** Iterate through the collected locations and calculate the 
travel time from Gotham (40.7128° N, 74.0060° W) to each location using the `calculate_cargo_travel_time` function.
6.  **Search for Supercar factories:** Use a web search to find websites listing supercar manufacturers.
7.  **Visit each relevant website:** Iterate through the search results, visiting the websites that seem promising.
8.  **Extract factory locations and their coordinates:** Extract the factory names and associated coordinates from 
each visited website.
9.  **Verify coordinates (if not provided):** For factories without readily available coordinates, use a web search
to find their latitude and longitude.
10. **Calculate travel times to supercar factories:** Iterate through the collected factories and calculate the 
travel time from Gotham (40.7128° N, 74.0060° W) to each factory using the `calculate_cargo_travel_time` function.
11. **Compare travel times:** Compare the travel times of Batman locations and supercar factories.
12. **Identify matching supercar factories:** Identify supercar factories with travel times similar (within a 
reasonable tolerance, perhaps +/- 10%) to the travel times of some Batman filming locations.
13. **Create a Pandas DataFrame:** Create a pandas DataFrame with Batman filming locations, their coordinates, and 
the cargo plane travel times.
14. **Final answer:** Provide the pandas DataFrame.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # 1. Search for Batman filming locations                                                                         
  batman_locations_search = web_search("Batman movie filming locations")                                           
  print(batman_locations_search)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The Dark Knight - Wikipedia](https://en.wikipedia.org/wiki/The_Dark_Knight)
Many interior locations for The Dark Knight were filmed on sets at Pinewood Studios, Buckinghamshire, and 
Cardington Airfield, Bedfordshire; these locations include the Bat Bunker, which took six weeks to build in a 
Cardington hangar.

[Batman Begins Filming Locations](https://tr.pinterest.com/ideas/batman-begins-filming-locations/948338290637/)
Batman Movie Filming Locations . Batman begins filming locations ve daha fazlası.

[Batman location hi-res stock photography and images - 
Alamy](https://www.alamy.com/stock-photo/batman-location.html)
Film location for 'The Batman ' Movie - 2020. Film location for 'The Batman ' Movie - 2020 Stock Photo Columns of 
St George's Hall and Benjamin Disraeli Statue on steps, Liverpool, Merseyside, England.

[Famous DC Movie Filming Locations You Should Know | 
TikTok](https://www.tiktok.com/@emilesam/video/7479133255292275990)
“How many of these filming Locations do you recognise from DC Movies ?!?”. Discover iconic filming locations from 
DC movies in this engaging video. Test your knowledge and see how many you recognize! #edit #vfx #loop.

[Batman movie filming to close downtown... | Crain's Chicago 
Business](https://www.chicagobusiness.com/article/20070719/NEWS/200025696/batman-movie-filming-to-close-downtown-st
reets)
(Crain's) -- Filming for "The Dark Knight" will prompt the closing of some downtown streets through Sunday. Filming
for "The Dark Knight" has been taking place in Chicago since June. Chicago stood in for the series' fictional 
Gotham City during the previous film , " Batman Begins."

[Is The Joker Actually In Love With Batman ? | GIANT FREAKIN 
ROBOT](https://www.giantfreakinrobot.com/ent/joker-love-batman.html)
The entire plot of the Lego Batman movie is driven by Joker wanting to hear that he is Batman ’s “greatest enemy,” 
complete with innuendo-filled dialogue, such as Batman ’s quippy “I like to fight around” when talking about 
Superman.

[The batman filming locations , The Batman filming locations](https://2j1nn.magna4thofjuly.org/)
Here are 'The Batman ' 2022 filming locations you can visit in real life. Among the regular attractions like 
Central Park, Empire State Building, Statue of Liberty and others, don’t forget to spot these revered Batman movie 
locations .

[James Gunn Shares DCU Filming Plans After Superman 
Success](https://screenrant.com/james-gunn-dc-universe-filming-plans-revealed/)
James Gunn opens up on future DC Universe movies and TV shows that are looking to start filming after Superman. 
Several DCU movies and TV shows are in various stages of development at DC Studios, along with Matt Reeves' The 
Batman universe, as a sequel is in the works with...

[Wanna Be in the Batman Movie ? It’ll Be Filming In Lansing Next 
Month](https://witl.com/wanna-be-in-the-batman-movie-itll-be-filming-in-lansing-next-month/)
The worst kept secret in Michigan is that the " Batman vs Superman" movie will be filming here in Lansing four 
weeks from now. And YOU might be one of the extras they're looking for. They've already been filming in Detroit for
the last month.

[Le location di Batman Begins – Film Locations for Batman 
Begins](https://batmancrimesolver.wordpress.com/2012/10/26/le-location-di-batman-begins/)
Secondo appuntamento in questa fantastica rubrica riguardante le location dei film di Batman , oggi andremo a 
scoprire i luoghi famosi all’interno del primo film di Christopher Nolan, Batman B…

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  from math import radians, sin, cos, sqrt, atan2                                                                  
                                                                                                                   
  # Coordinates for Gotham (40.7128° N, 74.0060° W)                                                                
  gotham_lat = 40.7128                                                                                             
  gotham_lon = -74.0060                                                                                            
                                                                                                                   
  # Function to calculate distance using Haversine formula                                                         
  def haversine(lat1, lon1, lat2, lon2):                                                                           
      R = 6371  # Radius of Earth in kilometers                                                                    
      lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])                                              
      dlon = lon2 - lon1                                                                                           
      dlat = lat2 - lat1                                                                                           
      a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2                                              
      c = 2 * atan2(sqrt(a), sqrt(1 - a))                                                                          
      distance = R * c                                                                                             
      return distance                                                                                              
                                                                                                                   
  # Function to calculate cargo travel time                                                                        
  def calculate_cargo_travel_time(origin_coords, destination_coords, cruising_speed_kmh=750):                      
      """Calculates the travel time for a cargo plane.  Returns travel time in hours."""                           
      lat1, lon1 = origin_coords                                                                                   
      lat2, lon2 = destination_coords                                                                              
      distance_km = haversine(lat1, lon1, lat2, lon2)                                                              
      travel_time_hours = distance_km / cruising_speed_kmh                                                         
      return travel_time_hours                                                                                     
                                                                                                                   
  # Known Batman locations                                                                                         
  batman_locations = [                                                                                             
      {"location": "Pinewood Studios, Buckinghamshire", "coordinates": None},                                      
      {"location": "Cardington Airfield, Bedfordshire", "coordinates": None},                                      
      {"

Execution logs:
Could not find coordinates for Pinewood Studios, Buckinghamshire
Found coordinates for Cardington Airfield, Bedfordshire: (52.1061, -0.4171)
Found coordinates for Chicago: (42.0411, -87.7102)
Found coordinates for St George's Hall, Liverpool: (53.409007, -2.980063)
Travel time to Cardington Airfield, Bedfordshire: 9.11 hours
Travel time to Chicago: 2.69 hours
Travel time to St George's Hall, Liverpool: 8.81 hours
Batman Locations DataFrame:
                            location             coordinates  \
0  Pinewood Studios, Buckinghamshire                    None   
1  Cardington Airfield, Bedfordshire      (52.1061, -0.4171)   
2                            Chicago     (42.0411, -87.7102)   
3        St George's Hall, Liverpool  (53.409007, -2.980063)   

   travel_time_hours  
0                NaN  
1               9.11  
2               2.69  
3               8.81  
Could not find coordinates for Pagani factory
Could not find coordinates for Ferrari factory
Could not find coordinates for Lamborghini factory
Could not find coordinates for McLaren factory

No matching supercar factories found.

Final answer:                             location             coordinates  \
0  Pinewood Studios, Buckinghamshire                    None   
1  Cardington Airfield, Bedfordshire      (52.1061, -0.4171)   
2                            Chicago     (42.0411, -87.7102)   
3        St George's Hall, Liverpool  (53.409007, -2.980063)   

   travel_time_hours  
0                NaN  
1               9.11  
2               2.69  
3               8.81  

[Step 8: Duration 14.68 seconds| Input tokens: 63,658 | Output tokens: 5,729]

In [ ]:
print(detailed_report)

                            location             coordinates  \
0  Pinewood Studios, Buckinghamshire                    None   
1  Cardington Airfield, Bedfordshire      (52.1061, -0.4171)   
2                            Chicago     (42.0411, -87.7102)   
3        St George's Hall, Liverpool  (53.409007, -2.980063)   

   travel_time_hours  
0                NaN  
1               9.11  
2               2.69  
3               8.81  


In [5]:
web_agent = CodeAgent(
    model=model2,
    tools=[
        GoogleSearchTool(provider="serper"),
        # DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)


In [6]:
import time

@tool
def sleeps(
    sleep_time: Optional[int] = 10,
) -> int:
    """
    Sleeps for a predefined number of seconds. Default value is 10 seconds.

    Args:
        sleep_time: Optional integer for number of seconds to sleep (defaults to 10 seconds)

    Returns:
        int: Sleep time in seconds

    Example:
        >>> # Sleep 5 seconds
        >>> result = sleep_time(5)
    """

    time.sleep(sleep_time)

    return sleep_time

print(sleeps())


10


In [7]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=big_model, #OpenAIServerModel("gpt-4o", max_tokens=8096), #big_model, #InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time, sleeps],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=50,
)

In [8]:
manager_agent.visualize()

CodeAgent | gemini-2.0-flash
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ sleeps                      │ Sleeps for a predefined number of     │ sleep_time (`integer`): Optional      │
│   │                             │ seconds. Default value is 10 seconds. │ integer for number of seconds to      │
│   │                             │                                       │ sleep (defaults to 10 seconds)        │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gemini/gemini-2.0-flash-lite
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel tim

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!
After every step sleep for 10 seconds

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")


In [ ]:
manager_agent.python_executor.state["fig"]